<a href="https://colab.research.google.com/github/andyherko/deep-img-classification/blob/main/build_train_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# main dependencies
import tensorflow as tf
import os
import cv2 
import imghdr
import numpy as np
from matplotlib import pyplot as plt
# deep learning
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout
# evaluate performance
from tensorflow.keras.metrics import Precision, Recall, BinaryAccuracy

In [ ]:
data_dir = '/content/drive/MyDrive/Colab Notebooks/input_data'
image_exts = ['jpeg', 'jpg', 'bmp', 'png']

## Remove unwanted images

In [ ]:
len(os.listdir('/content/drive/MyDrive/Colab Notebooks/input_data/sad'))

In [ ]:
for image_class in os.listdir(data_dir): 
    for image in os.listdir(os.path.join(data_dir, image_class)):
        image_path = os.path.join(data_dir, image_class, image)
        try: 
            img = cv2.imread(image_path)
            tip = imghdr.what(image_path)
            if tip not in image_exts: 
                print('Image not in ext list {}'.format(image_path))
                os.remove(image_path)
        except Exception as e: 
            print('Issue with image {}'.format(image_path))
            # os.remove(image_path)

## Load Data

In [ ]:
data = tf.keras.utils.image_dataset_from_directory('data')

## Scale data

In [ ]:
data = data.map(lambda x,y: (x/255, y))
batch = data.as_numpy_iterator().next()
batch[0].max()

In [ ]:
# fig, ax = plt.subplots(ncols=4, figsize=(20,20))
# for idx, img in enumerate(batch2[0][:4]):
#     ax[idx].imshow(img)
#     ax[idx].title.set_text(batch2[1][idx])

## Split Data

In [ ]:
train_size = int(len(data)*.7)
train_size

In [ ]:
val_size = int(len(data)*.2)
val_size

In [ ]:
test_size = int(len(data)*.1) + 2
test_size

In [ ]:
train_size + val_size + test_size == len(data)

In [ ]:
train = data.take(train_size)
val = data.skip(train_size).take(val_size)
test = data.skip(train_size+val_size).take(test_size)

## Build Deep Learning Model

In [ ]:
model = Sequential()

model.add(Conv2D(16, (3,3), 1, activation='relu', input_shape=(256,256,3)))
model.add(MaxPooling2D())
model.add(Conv2D(32, (3,3), 1, activation='relu'))
model.add(MaxPooling2D())
model.add(Conv2D(16, (3,3), 1, activation='relu'))
model.add(MaxPooling2D())
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile('adam', loss=tf.losses.BinaryCrossentropy(), metrics=['accuracy'])

In [ ]:
model.summary()

## Train

In [ ]:
logdir='logs'
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir)

hist = model.fit(train, epochs=20, validation_data=val, callbacks=[tensorboard_callback])

## Plot Performance